<h1 style="text-align:center">Build Regression Models</h1>
<h2 style="text-align:center">Applied Question 8</h2>
<p style="text-align:center">Robert Evans</p>
<p style="text-align:center">School of Technology & Engineering, National University</p>
<p style="text-align:center">DDS-8555: Predictive Analysis</p>
<p style="text-align:center">Dr. Mohammad Yavarimanesh</p>
<p style="text-align:center">January 26, 2025</p>

## Part A

In [1]:
import numpy as np

In [2]:
# set seed for repeatability
np.random.seed(42)

In [3]:
# Generate predictor X and noise vector epsilon
n = 100
X = np.random.normal(size=n)
epsilon = np.random.normal(size=n)

In [6]:
# Define coefficients
beta_0 = 2
beta_1 = 3
beta_2 = -1
beta_3 = 0.5

In [7]:
# Generate response vector Y
Y =  beta_0 + beta_1 * X + beta_2 * X**2 + beta_3 * X**3 + epsilon

## Part B

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from itertools import combinations

In [10]:
# Create a design matrix with X, X^2, ..., X^10
X_poly = np.column_stack([X**i for i in range(1, 11)])

In [13]:
def forward_stepwise_selection(X, Y):
    """
    Perform forward stepwise selection to select predictors for regression.

    Parameters:
        X (numpy.ndarray): The predictor matrix.
        Y (numpy.ndarray): The response vector.

    Returns:
        selected_model (LinearRegression): The final model selected.
        predictors (list): Indices of selected predictors.
    """
    predictors = []  # Start with no predictors selected
    min_cp = float("inf")  # Initialize the minimum Cp to a very large value
    selected_model = None  # Placeholder for the final selected model
    n = len(Y)  # Number of observations

    # Iterate over the number of predictors
    for _ in range(X.shape[1]):
        best_cp = float("inf")  # Initialize the best Cp for this iteration
        best_model = None  # Placeholder for the best model in this iteration
        best_predictors = None  # Placeholder for the best set of predictors

        # Test each predictor not already in the model
        for i in range(X.shape[1]):
            if i not in predictors:
                # Create a list of current predictors by adding the new predictor
                current_predictors = predictors + [i]
                
                # Extract the columns corresponding to the current predictors
                X_subset = X[:, current_predictors]
                
                # Fit a linear regression model on the subset of predictors
                model = LinearRegression().fit(X_subset, Y)
                y_pred = model.predict(X_subset)  # Generate predictions
                
                # Calculate Residual Sum of Squares (RSS)
                rss = sum((Y - y_pred) ** 2)
                
                # Compute Mallows' Cp (a measure of model quality)
                cp = rss + 2 * (len(current_predictors) + 1) * np.var(Y)
                
                # If this model is better (lower Cp), update the best model
                if cp < best_cp:
                    best_cp = cp
                    best_model = model
                    best_predictors = current_predictors

        # Update the selected predictors and the final model if Cp improves
        predictors = best_predictors
        if best_cp < min_cp:
            min_cp = best_cp
            selected_model = best_model

        # Print progress for each step
        print(f"Step {len(predictors)}: Added predictor {predictors[-1]} with Cp = {min_cp}")

    # Return the final selected model and predictors
    return selected_model, predictors

In [14]:
forward_model, forward_predictors = forward_stepwise_selection(X_poly, Y)
print("Forward Stepwise Predictors:", forward_predictors)
print("Coefficients:", forward_model.coef_)

Step 1: Added predictor 2 with Cp = 377.2770747678594
Step 2: Added predictor 0 with Cp = 251.31959202807815
Step 3: Added predictor 1 with Cp = 236.79744065727735
Step 4: Added predictor 9 with Cp = 236.79744065727735
Step 5: Added predictor 4 with Cp = 236.79744065727735
Step 6: Added predictor 8 with Cp = 236.79744065727735
Step 7: Added predictor 3 with Cp = 236.79744065727735
Step 8: Added predictor 7 with Cp = 236.79744065727735
Step 9: Added predictor 5 with Cp = 236.79744065727735
Step 10: Added predictor 6 with Cp = 236.79744065727735
Forward Stepwise Predictors: [2, 0, 1, 9, 4, 8, 3, 7, 5, 6]
Coefficients: [ 0.52836644  2.8642927  -0.79306238]


## Part C

In [15]:
def backward_stepwise_selection(X, Y):
    """
    Perform backward stepwise selection to select predictors for regression.

    Parameters:
        X (numpy.ndarray): The predictor matrix.
        Y (numpy.ndarray): The response vector.

    Returns:
        selected_model (LinearRegression): The final model selected.
        predictors (list): Indices of selected predictors.
    """
    # Start with all predictors initially included in the model
    predictors = list(range(X.shape[1]))
    min_cp = float("inf")  # Initialize the minimum Cp to a very large value
    selected_model = None  # Placeholder for the final selected model
    n = len(Y)  # Number of observations

    # Iterate until we have no predictors to remove
    while len(predictors) > 0:
        best_cp = float("inf")  # Initialize the best Cp for this iteration
        best_model = None  # Placeholder for the best model in this iteration
        best_predictors = None  # Placeholder for the best set of predictors

        # Test the model with each predictor removed
        for i in predictors:
            # Create a list of current predictors by removing one predictor
            current_predictors = [p for p in predictors if p != i]
            
            # Extract the columns corresponding to the current predictors
            X_subset = X[:, current_predictors]
            
            # Fit a linear regression model on the subset of predictors
            model = LinearRegression().fit(X_subset, Y)
            y_pred = model.predict(X_subset)  # Generate predictions
            
            # Calculate Residual Sum of Squares (RSS)
            rss = sum((Y - y_pred) ** 2)
            
            # Compute Mallows' Cp (a measure of model quality)
            cp = rss + 2 * (len(current_predictors) + 1) * np.var(Y)
            
            # If this model is better (lower Cp), update the best model
            if cp < best_cp:
                best_cp = cp
                best_model = model
                best_predictors = current_predictors

        # Update the selected predictors and the final model if Cp improves
        predictors = best_predictors
        if best_cp < min_cp:
            min_cp = best_cp
            selected_model = best_model

        # Print progress for each step
        print(f"Step {len(predictors)}: Removed predictor {i} with Cp = {min_cp}")

    # Return the final selected model and predictors
    return selected_model, predictors

In [16]:
backward_model, backward_predictors = backward_stepwise_selection(X_poly, Y)
print("Backward Stepwise Predictors:", backward_predictors)
print("Coefficients:", backward_model.coef_)

Step 9: Removed predictor 9 with Cp = 459.79967745107416
Step 8: Removed predictor 9 with Cp = 421.66246429349377
Step 7: Removed predictor 9 with Cp = 383.7795759122897
Step 6: Removed predictor 9 with Cp = 346.86094134618656
Step 5: Removed predictor 9 with Cp = 308.9371290662358
Step 4: Removed predictor 9 with Cp = 283.86508326048545
Step 3: Removed predictor 7 with Cp = 256.0500575519655
Step 2: Removed predictor 7 with Cp = 237.26390701196368
Step 1: Removed predictor 5 with Cp = 237.26390701196368


ValueError: Found array with 0 feature(s) (shape=(100, 0)) while a minimum of 1 is required by LinearRegression.